<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/sbert/sentence_vector_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets sentence_transformers torch

     |████████████████████████████████| 290 kB 7.3 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
     |████████████████████████████████| 1.1 MB 39.0 MB/s 
     |████████████████████████████████| 243 kB 51.5 MB/s 
     |████████████████████████████████| 132 kB 45.7 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 3.1 MB 37.7 MB/s 
     |████████████████████████████████| 3.3 MB 34.4 MB/s 
     |████████████████████████████████| 1.2 MB 24.4 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 596 kB 38.6 MB/s 
     |████████████████████████████████| 192 kB 44.4 MB/s 
     |████████████████████████████████| 271 kB 49.2 MB/s 
     |████████████████████████████████| 160 kB 48.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=9c6e04b61f18972ef19524a22414899cb99b78852ceb2c2aba9c85e73ba0c255
  Stored in 

In [2]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [4]:
# pytorch gpu 사용
import torch

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:',device)

True
학습을 진행하는 기기: cuda:0


In [5]:
model_name = "klue/roberta-base"
train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [6]:
datasets = load_dataset("klue", "sts")
datasets["train"][0]

Downloading:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [7]:
testsets = load_dataset("kor_nlu", "sts")
testsets["test"][0]

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.1k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset kor_nlu downloaded and prepared to /root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'filename': 2,
 'genre': 1,
 'id': 24,
 'score': 2.5,
 'sentence1': '한 소녀가 머리를 스타일링하고 있다.',
 'sentence2': '한 소녀가 머리를 빗고 있다.',
 'year': 6}

In [8]:
# 두 데이터 모두 0점에서 5점 사이의 값으로 유사도가 기록되었기 때문에, 0.0 ~ 1.0 스케일로 정규화
train_samples = []
dev_samples = []
test_samples = []

#KLUE STS 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]
    
    for example in examples:
        score = float(example["labels"]["label"]/5.0) #0.0~1.0 사이로 정규화
        
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )
        
        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [9]:
# KorSTS 내에서 테스트 데이터 예제 변환

for example in testsets["test"]:
    score = float(example["score"]) / 5.0
    
    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score
        )
    test_samples.append(inp_example)

In [10]:
train_samples[0].texts, train_samples[0].label
test_samples[0].texts, test_samples[0].label
dev_samples[0].texts, dev_samples[0].label

(['무엇보다도 호스트분들이 너무 친절하셨습니다.', '무엇보다도, 호스트들은 매우 친절했습니다.'], 0.9800000000000001)

In [11]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [14]:
# DataLoader 과 Loss 설정하기
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size
)

embedding_model = models.Transformer(model_name)

pooler = models.Pooling(
embedding_model.get_word_embedding_dimension(),
pooling_mode_mean_tokens=True,
pooling_mode_cls_token=False,
pooling_mode_max_tokens=False
)

model = SentenceTransformer(modules=[embedding_model, pooler])

train_loss = losses.CosineSimilarityLoss(model=model)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev"
)
warmup_steps = math.ceil(len(train_dataloader)*num_epochs*0.1) #10% of train_data warm up
logging.info(f"Warmup-steps:{warmup_steps}")


model.fit(
train_objectives=[(train_dataloader, train_loss)],
evaluator=evaluator,
epochs=num_epochs,
evaluation_steps=1000,
warmup_steps=warmup_steps,
output_path=model_save_path
)


Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

2021-11-22 04:29:44 - Use pytorch device: cuda
2021-11-22 04:29:44 - Warmup-steps:146


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-11-22 04:35:26 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2021-11-22 04:35:30 - Cosine-Similarity :	Pearson: 0.8703	Spearman: 0.8644
2021-11-22 04:35:30 - Manhattan-Distance:	Pearson: 0.8692	Spearman: 0.8619
2021-11-22 04:35:30 - Euclidean-Distance:	Pearson: 0.8700	Spearman: 0.8629
2021-11-22 04:35:30 - Dot-Product-Similarity:	Pearson: 0.8593	Spearman: 0.8495
2021-11-22 04:35:30 - Save model to output/training_klue_sts_klue-roberta-base-2021-11-22_03-59-06


...

2021-11-22 04:41:13 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2021-11-22 04:41:17 - Cosine-Similarity :	Pearson: 0.8836	Spearman: 0.8804
2021-11-22 04:41:17 - Manhattan-Distance:	Pearson: 0.8819	Spearman: 0.8763
2021-11-22 04:41:17 - Euclidean-Distance:	Pearson: 0.8827	Spearman: 0.8772
2021-11-22 04:41:17 - Dot-Product-Similarity:	Pearson: 0.8718	Spearman: 0.8658
2021-11-22 04:41:17 - Save model to output/training_klue_sts_klue-roberta-base-2021-11-22_03-59-06


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-11-22 04:47:00 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2021-11-22 04:47:04 - Cosine-Similarity :	Pearson: 0.8883	Spearman: 0.8871
2021-11-22 04:47:04 - Manhattan-Distance:	Pearson: 0.8888	Spearman: 0.8835
2021-11-22 04:47:04 - Euclidean-Distance:	Pearson: 0.8896	Spearman: 0.8846
2021-11-22 04:47:04 - Dot-Product-Similarity:	Pearson: 0.8779	Spearman: 0.8741
2021-11-22 04:47:04 - Save model to output/training_klue_sts_klue-roberta-base-2021-11-22_03-59-06


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-11-22 04:52:48 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2021-11-22 04:52:52 - Cosine-Similarity :	Pearson: 0.8894	Spearman: 0.8890
2021-11-22 04:52:52 - Manhattan-Distance:	Pearson: 0.8906	Spearman: 0.8849
2021-11-22 04:52:52 - Euclidean-Distance:	Pearson: 0.8918	Spearman: 0.8869
2021-11-22 04:52:52 - Dot-Product-Similarity:	Pearson: 0.8787	Spearman: 0.8744
2021-11-22 04:52:52 - Save model to output/training_klue_sts_klue-roberta-base-2021-11-22_03-59-06


In [15]:
torch.save(model, model_save_path + '/klue-roberta-ftned.pt')  # 전체 모델 저장